In [1]:
import geopandas as gpd
from shapely.geometry import LineString
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd

# Load original line layer with 'pct'
gdf = gpd.read_file("ortodromic_erope.gpkg")

# Prepare list to collect new segments
segments = []

# Function to generate color value between red and green based on position
cmap = cm.get_cmap('RdYlGn')  # Red to Green colormap
def get_color(position):
    rgba = cmap(position)
    hex_color = mcolors.to_hex(rgba)
    return hex_color

# Split each line into N segments and assign color
N = 100  # Number of segments
for _, row in gdf.iterrows():
    line = row.geometry
    pct = row.pct if pd.notnull(row.pct) else 0

    if not isinstance(line, LineString) or line.length == 0:
        continue

    for i in range(N):
        start_frac = i / N
        end_frac = (i + 1) / N

        start_point = line.interpolate(start_frac, normalized=True)
        end_point = line.interpolate(end_frac, normalized=True)

        seg = LineString([start_point, end_point])
        
        if start_frac <= pct / 100:
            color_value = get_color(start_frac)
        else:
            color_value = get_color(pct / 100)

        segments.append({
            'origin': row['origin'],
            'destination': row['destination'],
            'value': row['value'],
            'pct': pct,
            'position': start_frac,
            'color': color_value,
            'geometry': seg
        })

# Convert to GeoDataFrame
gdf_segments = gpd.GeoDataFrame(segments, crs=gdf.crs)

# Export
output_path = "ortodromic_erope_gradient_segments.gpkg"
gdf_segments.to_file(output_path, driver="GPKG", layer="gradient_lines")

print("✅ Export done:", output_path)


C:\Users\michal\AppData\Local\Temp\ipykernel_15756\3864526538.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('RdYlGn')  # Red to Green colormap


✅ Export done: ortodromic_erope_gradient_segments.gpkg
